In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image 
import os



In [2]:
image_path = 'dataset/color'

#convert the images and save them in a array. 
color_X = []
color_y = []
for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = image.convert('RGB')
                #image = image.resize((width, height))
                color_X.append(np.array(image))
                color_y.append(os.path.basename(folder_name))
            else:
                pass
    else:
        pass

In [3]:
# convert the images and labels into arrays
color_X_array = np.array(color_X)
color_y_array  = np.array(color_y)

In [4]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(color_X_array, color_y_array, test_size=0.2, random_state=42)

In [5]:
# getting a small portion of the set for faster training 20% of the total data

X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

In [21]:
y_test_one.shape

(544, 38)

In [6]:
# Encoding the label into integers

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y_train_le = le.fit_transform(y_train_small)
y_test_le = le.transform(y_test_small)


In [7]:
# One hot encoding of the labels

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_train_re = y_train_le.reshape(-1, 1)
y_test_re = y_test_le.reshape(-1, 1)
y_train_one = onehot_encoder.fit_transform(y_train_re)
y_test_one = onehot_encoder.transform(y_test_re)

In [10]:
# Model 1: CNN 2 convolution layers

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model1 = Sequential()

model1.add(Conv2D(64,kernel_size=(3,3),activation="relu", padding="same",
                  kernel_regularizer=regularizers.l2(0.00001), input_shape=(256, 256, 3)))
model1.add(MaxPooling2D(pool_size=6, strides=2, padding='same'))
model1.add(Dropout(0.25))

model1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=6, strides=2, padding='same'))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(128,activation="relu")) 
model1.add(Dropout(0.5))
model1.add(Dense(38,activation="softmax"))

In [15]:
# Compile and fit Model 1
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1_history = model1.fit(X_train_small, y_train_one, batch_size=128, epochs=10, validation_data=(X_test_small, y_test_one))

Epoch 1/10
68/68 [==============================] - 65s 941ms/step - loss: 3.1016 - accuracy: 0.1524 - val_loss: 3.3029 - val_accuracy: 0.1121
Epoch 2/10
68/68 [==============================] - 63s 930ms/step - loss: 3.0704 - accuracy: 0.1469 - val_loss: 3.3012 - val_accuracy: 0.1140
Epoch 3/10
68/68 [==============================] - 63s 930ms/step - loss: 3.0206 - accuracy: 0.1680 - val_loss: 3.2893 - val_accuracy: 0.1213
Epoch 4/10
68/68 [==============================] - 2203s 33s/step - loss: 2.9912 - accuracy: 0.1768 - val_loss: 3.2778 - val_accuracy: 0.1250
Epoch 5/10
68/68 [==============================] - 63s 925ms/step - loss: 2.9328 - accuracy: 0.1934 - val_loss: 3.3284 - val_accuracy: 0.1140
Epoch 6/10
68/68 [==============================] - 62s 907ms/step - loss: 2.8967 - accuracy: 0.1948 - val_loss: 3.3717 - val_accuracy: 0.1324
Epoch 7/10
68/68 [==============================] - 62s 918ms/step - loss: 2.8186 - accuracy: 0.2284 - val_loss: 3.3441 - val_accuracy: 0.1324

In [14]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

# Model 2: CNN 1 convolution layer

model_conv = Sequential()

model_conv.add(Conv2D(filters = 64,
                     input_shape = (256,256,3),
                     kernel_size = (5,5),
                     padding = 'same',
                     activation = 'relu'))

model_conv.add(MaxPooling2D(pool_size = (2, 2)))

model_conv.add(Dropout(rate = 0.2))

model_conv.add(Flatten())

model_conv.add(Dense(units = 256,
                    activation = 'relu'))

model_conv.add(Dense(units = 38,
                     activation='softmax'))

model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

modelconv_history = model_conv.fit(X_train_small, y_train_one, batch_size=64, epochs=20, validation_data=(X_test_small, y_test_one))



Epoch 1/20
136/136 [==============================] - 367s 3s/step - loss: 1065.1208 - accuracy: 0.1380 - val_loss: 3.1113 - val_accuracy: 0.1758
Epoch 2/20
136/136 [==============================] - 368s 3s/step - loss: 2.2649 - accuracy: 0.4101 - val_loss: 2.8050 - val_accuracy: 0.2876
Epoch 3/20
136/136 [==============================] - 368s 3s/step - loss: 1.0936 - accuracy: 0.7081 - val_loss: 3.1836 - val_accuracy: 0.3143
Epoch 4/20
136/136 [==============================] - 368s 3s/step - loss: 0.5907 - accuracy: 0.8451 - val_loss: 5.4903 - val_accuracy: 0.3645
Epoch 5/20
136/136 [==============================] - 366s 3s/step - loss: 0.4937 - accuracy: 0.8804 - val_loss: 5.4495 - val_accuracy: 0.3083
Epoch 6/20
136/136 [==============================] - 365s 3s/step - loss: 0.5211 - accuracy: 0.8828 - val_loss: 6.1651 - val_accuracy: 0.3507
Epoch 7/20
136/136 [==============================] - 369s 3s/step - loss: 0.3184 - accuracy: 0.9251 - val_loss: 6.9071 - val_accuracy: 0.3

In [13]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

# Model 3:  CNN 1 convolution layer

model1 = Sequential()

model1.add(Conv2D(filters = 64,
                     input_shape = (256,256,3),
                     kernel_size = (5,5),
                     padding = 'same',
                     activation = 'relu'))

model1.add(MaxPooling2D(pool_size = (2, 2)))

model1.add(Dropout(rate = 0.2))

model1.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
model1.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model1.add(Dropout(0.2))

model1.add(Flatten())

model1.add(Dense(units = 128,
                    activation = 'relu'))

model1.add(Dense(units = 38,
                     activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model1_history = model1.fit(X_train_small, y_train_one, batch_size=64, epochs=20, validation_data=(X_test_small, y_test_one))



Epoch 1/20
136/136 [==============================] - 477s 3s/step - loss: 62.2484 - accuracy: 0.1140 - val_loss: 3.4019 - val_accuracy: 0.1275
Epoch 2/20
136/136 [==============================] - 4378s 32s/step - loss: 3.3187 - accuracy: 0.1214 - val_loss: 3.2661 - val_accuracy: 0.1238
Epoch 3/20
136/136 [==============================] - 471s 3s/step - loss: 3.2191 - accuracy: 0.1296 - val_loss: 3.2887 - val_accuracy: 0.1431
Epoch 4/20
136/136 [==============================] - 3393s 25s/step - loss: 3.1130 - accuracy: 0.1609 - val_loss: 3.2397 - val_accuracy: 0.1362
Epoch 5/20
136/136 [==============================] - 470s 3s/step - loss: 2.8742 - accuracy: 0.2179 - val_loss: 3.2624 - val_accuracy: 0.1440
Epoch 6/20
136/136 [==============================] - 467s 3s/step - loss: 2.6669 - accuracy: 0.2682 - val_loss: 3.4377 - val_accuracy: 0.1445
Epoch 7/20
136/136 [==============================] - 472s 3s/step - loss: 2.4726 - accuracy: 0.3189 - val_loss: 3.5772 - val_accuracy: 0